                                                   How to run
First you run the cell for the dataset generation, which will save everything in binary 
Then run cells "From binary to indexes" to save everything in the indexes file
Each cell requires around 2 min of waiting(

In [ ]:
import numpy as np
np.random.seed(1) # this will fix the seed. (every time the random function will give you the same result, so
#the dataset is the same) YOU DON'T HAVE TO RUN IT, ONLY IF YOU WANT

# From binary to indexes

In [92]:
with open("binary"+str(N)+".txt") as f:
    lines = f.readlines()

In [35]:
len(lines)

25000

In [93]:
# frozen bits correspond to 1 !!!

# extract positions from the binary representation
import numpy as np

lines_num = len(lines) # how many lines from the dataset are needed ALL->lines_num = len(lines)
digits = []

for i in range(lines_num):
    code = np.array([int(s) for s in lines[i].split() if s.isdigit()])# ignore delimeters and save as an array
    
    frozen_pos = np.where(np.array(code) != 0)[0] 
    info_pos = np.where(np.array(code) == 0)[0]
    
    bits_list = info_pos.tolist() + frozen_pos.tolist() # first half is info, the other is frozen
    #bits_list = info_pos.tolist() + ["||"] + frozen_pos.tolist()
    
    textfile = open("indexes"+str(N)+".txt", "a")
    for element in bits_list:
        textfile.write(str(element) + " ")
    textfile.write("\n")
    
textfile.close()
    

# From indexes to binary

In [3]:
with open("indexes"+str(N)+".txt") as f:#dataset with info(first half) and frozen(second half)
    lines = f.readlines()

In [4]:
import numpy as np

N = 256

for i in range(len(lines)):
    code = np.array([int(s) for s in lines[i].split() if s.isdigit()])# ignore delimiters and save as an array
    
    
    info_pos = code[0:int(N/2)] # first half is info
    arr = np.ones(N, dtype=int)
    arr[info_pos] = 0 # info correspond to 0
    
    bits_list = arr.tolist()
    
    textfile = open("binary"+str(N)+".txt", "a")
    for element in bits_list:
        textfile.write(str(element) + " ")
    textfile.write("\n")
    
textfile.close()

# Dataset generation from mat file


In [1]:
###########################################
import scipy.io
import numpy as np

N = 1024

name = 'N'+str(N)+'_frozenSet_GaussianApprox_v3.mat'
m = scipy.io.loadmat(name)["err_prob"] 
mat = m.T[0]

indexes = np.argsort(mat) #from the most reliable to the least

In [4]:
mat

array([5.00000000e-001, 5.00000000e-001, 5.00000000e-001, ...,
       4.72110911e-209, 4.75269888e-111, 0.00000000e+000])

In [3]:
indexes

array([1023, 1021, 1019, ...,  160,  134,    0], dtype=int64)

In [6]:
size = 19*(10**3)

r = 64 # permutation range

filename = "binary"+str(N)+"_with_duplicates.txt"

for k in range(size):
    
    if k == 0: # first sample is GA 
        binary = np.ones(N, dtype = int)
        binary[(np.array(indexes[0:int(N/2)]))] = 0 #info bits ->0
        bits_list = binary.tolist()
        
        textfile = open(filename, "a")
        for element in bits_list:
            textfile.write(str(element) + " ")
        textfile.write("\n")
        continue
    
    temp = indexes.copy()
    np.random.shuffle(temp[N//2-r:N//2+r])
    binary = np.ones(N, dtype = int)
    binary[(np.array(temp[0:int(N/2)]))] = 0
    
    bits_list = binary.tolist()
    
    textfile = open(filename, "a")
    for element in bits_list:
        textfile.write(str(element) + " ")
    textfile.write("\n") 
    #print(k)
    
num_lines = sum(1 for line in open(filename))

print(num_lines)
textfile.close()

18999


In [7]:
#check # of unique
from collections import OrderedDict

seen = OrderedDict()
for line in open(filename):
    line = line.strip() #remove ' '
    seen[line] = seen.get(line, 0) + 1

#print("\n".join([k for k,v in seen.items() if v == 1]))
print("Unique rows: "+str(len(seen)))

Unique rows: 19000


In [24]:
# clean file
openFile = open(filename, "r") 
writeFile = open("binary"+str(N)+".txt", "w") 
#Store traversed lines
tmp = set() 
for txtLine in openFile: 
#Check new line
    if txtLine not in tmp: 
        writeFile.write(txtLine) 
#Add new traversed line to tmp 
        tmp.add(txtLine)
    
num_lines = sum(1 for line in open("binary"+str(N)+".txt"))

print(num_lines)    
openFile.close() 
writeFile.close()

18996


#  Gaussian approximation (if you need just it in the file)


In [37]:
import scipy.io
import numpy as np

N = 256 #64#256 #1024  
name = 'N'+str(N)+'_frozenSet_GaussianApprox.mat'
m = scipy.io.loadmat(name)["indexes"] #save the frozen bits
mat = m.T[0] - 1

#frozen_pos = mat.tolist() #wrong
info_pos = mat.tolist()

arr = np.arange(0, N)
# info_pos = arr.tolist() #wrong
frozen_pos = arr.tolist()

#for i in frozen_pos: #wrong
#    info_pos.remove(i)
#frozen_pos.sort()

for i in info_pos:
    frozen_pos.remove(i)
info_pos.sort()

In [38]:
#create an array of zeroes and ones 
binary = np.zeros(N, dtype = int)
binary[(np.array(frozen_pos))] = 1 # frozen bits correspond to 1 !!!

In [39]:
filename = "GA_"+str(N)+"_binary.txt"

bits_list = binary.tolist()
    
textfile = open(filename, "a")
for element in bits_list:
    textfile.write(str(element) + " ")
    
textfile.close()

In [40]:
with open('GA_'+str(N)+'_binary.txt') as f:
    lines = f.readlines()
    
lines_num = 1 # how many lines from the dataset are needed
digits = []

for i in range(lines_num):
    code = np.array([int(s) for s in lines[i].split() if s.isdigit()])# ignore delimeters and save as an array
    
    frozen_pos = np.where(np.array(code) != 0)[0] # not to start from zero
    info_pos = np.where(np.array(code) == 0)[0]
    
    bits_list = info_pos.tolist() + frozen_pos.tolist() # first half is info, the other is frozen
    #bits_list = info_pos.tolist() + ["||"] + frozen_pos.tolist()
    
    textfile = open("GA_"+str(N)+"_indexes.txt", "a")
    for element in bits_list:
        textfile.write(str(element) + " ")
    textfile.write("\n")
    
textfile.close()

In [90]:
num_lines = sum(1 for line in open(filename))
num_lines

25000